In [1]:
!pip install spotipy

     |████████████████████████████████| 62 kB 3.5 MB/s  eta 0:00:01
     |████████████████████████████████| 138 kB 64.0 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: requests
    Found existing installation: requests 2.11.1
    Uninstalling requests-2.11.1:
      Successfully uninstalled requests-2.11.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

two1 3.10.9 requires mnemonic==0.13, but you'll have mnemonic 0.18 which is incompatible.
two1 3.10.9 requires protobuf==3.0.0a3, but you'll have protobuf 3.6.1 which is incompatible.
two1 3.10.9 requires requests<=2.11.1, but you'll have requ

In [26]:
import numpy as np
import pandas as pd
import sys
import spotipy
import spotipy.util as util

In [27]:
# spotify = spotipy.Spotify()
# results = spotify.search(q='artist:' + "Beyonce", type='artist')
# print(results)

SPOTIPY_CLIENT_ID="70c49e55eae64f0a82b313830c6cebf3"
SPOTIPY_CLIENT_SECRET="cb5063c70a514acebca3481a37f15e01"
SPOTIPY_REDIRECT_URI = "http://localhost"
username='axw2001d'
scope="user-library-read"
token = util.prompt_for_user_token(username, 
                                   scope, 
                                   client_id=SPOTIPY_CLIENT_ID, 
                                   client_secret=SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri=SPOTIPY_REDIRECT_URI
                                  )

Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.


Enter the URL you were redirected to: http://localhost/?code=AQDJx-ELThmu-gQ0ioiEnXxMv1t3_Yq2Zj4-sde-67Ort3rLa60qGyKlRTADM0-PZSqcvdwdSqO2ffufRjmGoQ68ZVTVqNPX5hC_aFtQsDA2Uso6GSZKTTpo0w_8IB8OKxJdRH6QkUqycA8QLlbeqLDyBvo44TbcX3VMTKglUl7LEDMWBRJF


In [59]:
# def get_playlist_tracks(username,playlist_id):
#     results = sp.user_playlist_tracks(username,playlist_id)
#     tracks = results['items']
#     while results['next']:
#         results = sp.next(results)
#         tracks.extend(results['items'])
#     return tracks

def get_saved_songs(token):
    
    features_list = ["artist", "album", "album_release_date", "track_name", "track_id", "explicit", "popularity", "danceability", "energy", "key", "loudness", "mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    df = pd.DataFrame(columns = features_list)
    
    if token:
        sp = spotipy.Spotify(auth=token)
        results = sp.current_user_saved_tracks()
        saved_songs = results['items']
        
        num_pages = 0
        while results['next'] and num_pages < 10:
            results = sp.next(results)
            saved_songs.extend(results['items'])
            num_pages += 1
        
        for item in saved_songs:
            track = item['track']
            features = {}
            
            # get metadata
            features['artist'] = track['album']['artists'][0]['name']
            features['album_release_date'] = track['album']['release_date']
            features['album'] = track['album']['name']
            features['track_name'] = track['name']
            features['track_id'] = track['id']
            features['explicit'] = track['explicit']
            features['popularity'] = track['popularity']
            
            # get audio features
            audio_features = sp.audio_features(features['track_id'])[0]
            for feature in features_list[7:]: # this splice is hard coded, must update if changing features
                features[feature] = audio_features[feature]
            
            # concat dataframes
            track_df = pd.DataFrame(features, index=[0])
            df = pd.concat([df, track_df], ignore_index=True)
            
        return df
    else:
        return ("Can't get token for", username)

albertSongs = get_saved_songs(token)
albertSongs

,artist,album,album_release_date,track_name,track_id,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,88rising,Shang-Chi and The Legend of The Ten Rings: The...,2021-09-03,In The Dark (with Jhené Aiko),0zaoWwS8RpE3LSDdmkg8TC,False,65,0.686,0.654,6,-6.700,1,0.0936,0,0.1080,0.663,98.077,161147,4
1,tobi lou,tobi lou and the Loop - EP,2018-04-27,Buff Baby,1F1QmI8TMHir9SUFrooq5F,True,65,0.734,0.765,5,-6.047,1,0.0571,0,0.1010,0.512,156.040,215385,4
2,MAX,Colour Vision,2020-09-18,Blueberry Eyes (feat. SUGA of BTS),6sI8Q7TTJeA9exw6hxEqq3,False,69,0.746,0.611,0,-7.034,0,0.1010,0.000922,0.1330,0.411,149.967,172245,4
3,PRETTYMUCH,Up to You (feat. NCT DREAM),2019-11-21,Up to You (feat. NCT DREAM),58LUjGxK8NF8VD7KxRvONb,False,57,0.719,0.803,5,-4.714,0,0.1350,0,0.1470,0.719,115.925,160578,4
4,Lauv,Without You,2020-07-10,Mine (You Can't Find Love in Mollywood),6uTDvnX30fst2aaHWuD9kI,True,49,0.734,0.499,1,-7.453,0,0.0318,5.28e-06,0.0933,0.382,126.053,206133,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,CODE KUNST,rain bird,2018-06-10,rain bird,3JAlslgvIfvpbEqk94afpd,False,46,0.724,0.637,9,-7.653,0,0.0818,0,0.0918,0.369,129.921,224308,4
216,YG,STAY DANGEROUS,2018-08-03,TOO COCKY,7GG1QIQ547taCwl1uLaTf5,True,46,0.925,0.354,5,-8.163,1,0.1300,0,0.3340,0.405,100.026,172773,4
217,YG,4REAL 4REAL,2019-05-24,Stop Snitchin,5IjRKZlpKVBBRn0COL7pik,True,61,0.842,0.743,7,-4.430,1,0.3290,3.69e-06,0.1510,0.622,180.917,149880,4
218,H.E.R.,Slide,2019-09-27,Slide,2rTnVB1bvwxHtaIl4uVu7f,True,68,0.832,0.469,10,-9.141,0,0.3390,8.26e-06,0.2070,0.197,97.023,238321,4


In [60]:
albertSongs.shape

(220, 19)

In [41]:
# pd.DataFrame(pd.json_normalize(track))

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,161146,False,https://api.spotify.com/v1/tracks/0zaoWwS8RpE3...,0zaoWwS8RpE3LSDdmkg8TC,False,In The Dark (with Jhené Aiko),65,...,2kAqjStKcwlDD59H0llhGC,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Shang-Chi and The Legend of The Ten Rings: The...,2021-09-03,day,18,album,spotify:album:2kAqjStKcwlDD59H0llhGC,USUM72113050,https://open.spotify.com/track/0zaoWwS8RpE3LSD...


In [52]:
# track.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [57]:
# track

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1AhjOkOLkbHUfcHDSErXQs'},
    'href': 'https://api.spotify.com/v1/artists/1AhjOkOLkbHUfcHDSErXQs',
    'id': '1AhjOkOLkbHUfcHDSErXQs',
    'name': '88rising',
    'type': 'artist',
    'uri': 'spotify:artist:1AhjOkOLkbHUfcHDSErXQs'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/2kxP07DLgs4xlWz8YHlvfh'},
    'href': 'https://api.spotify.com/v1/artists/2kxP07DLgs4xlWz8YHlvfh',
    'id': '2kxP07DLgs4xlWz8YHlvfh',
    'name': 'NIKI',
    'type': 'artist',
    'uri': 'spotify:artist:2kxP07DLgs4xlWz8YHlvfh'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/540vIaP2JwjQb9dm3aArA4'},
    'href': 'https://api.spotify.com/v1/artists/540vIaP2JwjQb9dm3aArA4',
    'id': '540vIaP2JwjQb9dm3aArA4',
    'name': 'DJ Snake',
    'type': 'artist',
    'uri': 'spotify:artist:540vIaP2JwjQb9dm3aArA4'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
 